In [1]:
import ipywidgets 
import glob
import json
import pandas as pd
from ipyleaflet import Map, Marker, CircleMarker, Circle, Popup
from IPython.display import display 
from twilio.rest import Client
import datetime

#takeawayFolderPath = '/home/hd2900/Documents/Python/POS/takeawayOrders'
takeawayFolderPath = '/Users/jianxiongwu/Documents/Python/Github/POS/takeawayOrders'
class GeoMapping:
    def __init__(self, takeawayFolderPath, homeCoordinate, deliveryRadius):
        #Get credential for sending the sms
        self.cred = self.readJsonFile('/etc/config.json')
        self.takeawayFolderPath = takeawayFolderPath
        self.homeCoordinate = homeCoordinate
        self.deliveryRadius = int(deliveryRadius) #The deliver radius is given in meters
        self.currentSelectedOrder = None
        self.refreshJson()
        self.getOrderDataFrame()
        self.buildGUI()
        self.displayTable()
        self.displayMap()
    
    def buildGUI(self):
        self.btn_update = ipywidgets.Button(description = 'Refresh')
        self.btn_update.on_click(self.update_btn)
        self.tableOutput1 = ipywidgets.Output()
        self.btn_orderComplete = ipywidgets.Button(description = 'Complete', button_style = 'danger')
        self.btn_orderComplete.on_click(self.orderComplete)
        self.mapOutput = ipywidgets.Output()
        
        #Create a segment for SMS
        self.smsHTML = ipywidgets.HTML(
            value="",
            placeholder='',
            description='',
        )
        
        self.delayLabel = ipywidgets.Label(value = 'Delay minutes')
        self.delayMinutes = ipywidgets.BoundedIntText(
            value=0,
            min=0,
            max=60,
            step=5,
            disabled=False,
            layout = ipywidgets.Layout(width = '100px'))
        
        self.btn_sendSMS = ipywidgets.Button(description = 'Send SMS', button_style = 'danger')
        self.btn_sendSMS.on_click(self.sendSMS_btn)
        
        self.Vbox1 = ipywidgets.VBox([self.delayLabel, self.delayMinutes])   
        self.HBox = ipywidgets.HBox([self.smsHTML, self.Vbox1, self.btn_sendSMS])
        
        self.VBox2 = ipywidgets.VBox([self.btn_update, self.tableOutput1, self.HBox, self.mapOutput, self.btn_orderComplete,])
        display(self.VBox2)
        
    def displayMap(self):
        #Centering the map at home coordinate
        self.map = Map(center= self.homeCoordinate, zoom=12)
            
        #Add a red circle marker to show home. This is not the delivery 
        circleMarker = CircleMarker()
        circleMarker.location = self.homeCoordinate
        circleMarker.radius=5
        circleMarker.color = "red"
        circleMarker.fill_color="red"
        self.map.add_layer(circleMarker)
            
        #Add delivery radius ring
        circle = Circle()
        circle.radius = self.deliveryRadius
        circle.location= self.homeCoordinate
        circle.color= "blue"
        circle.fill=False
        self.map.add_layer(circle)
            
        #Take time into account and plot the maps in different color. The color order
        self.markers = list()
        
        for orderid in self.orderDF['Order Id']:
            latitude, longitude, deadline = self.getGeoCoordinateFromOrderId(orderid)
                
            #Plot the coordinates in map
            marker = Marker(
                location = [latitude, longitude], 
                title = f"{orderid} {deadline}", 
                draggable = False,
                z_index_offset = 30,
                rise_on_hover = True)
            
            popupMessage = ipywidgets.HTML()
            popupMessage.value = f"<b>{orderid}, {deadline}</b>"
            
            marker.popup = popupMessage
            marker.on_click(self.markerClicked)

            self.map.add_layer(marker)
            self.markers.append(marker)
                
        with self.mapOutput:
            self.mapOutput.clear_output()
            display(self.map)
    
    def markerClicked(self, **kwargs):
        #The only way to identify which marker has been clicked is through the gps coordinate
        
        #Loop through the coordinates to find the exact match 
        print(kwargs)
        order = self.getOrderIdFromGeoCoordinate(kwargs['coordinates'])
        
        #Insert the clicked information in the html box
        self.smsHTML.value = f'''<h1>Order id : {order['id']}
        <br>Deadline {order['deadline']}</h1> '''
        
        #Reset the delay time to 0
        self.delayMinutes.value = 0
        
        self.currentSelectedOrder = order
        
    def displayTable(self):
        with self.tableOutput1:
            self.tableOutput1.clear_output()
            display(self.orderDF)
    
    def orderComplete(self,b):
        
        if not self.currentSelectedOrder:
            return
        
        #Find the json file with the order id and write a new key in the dictionary
        filePath = self.getJsonFilePathFromOrderId(self.currentSelectedOrder['id'])
        
        order = self.readJsonFile(filePath)
        order['Complete'] = True
        
        self.saveJsonFile(order, filePath)
        self.update()
    
    def getJsonFilePathFromOrderId(self, orderId):
        '''
        Given the order Id integer, this method returns the json file full path
        '''
        files = glob.glob(self.takeawayFolderPath + '/*.json')
        for fileName in files:
            order = self.readJsonFile(fileName)
            
            if order['order']['id'] == orderId:
                return fileName
            
    def sendSMS_btn(self, b):
        if not self.currentSelectedOrder or self.delayMinutes.value == 0:
            return
        
        #Send the sms
        newDeadlineTime = self._getNewDeadline(delayMinutes = self.delayMinutes.value, currentDeadline = self.currentSelectedOrder['deadline'])
        print(newDeadlineTime)
        smsMsg = self._genSMSMessage(customerName = self.currentSelectedOrder['fullName'], newDeadlineTime = newDeadlineTime)
        
        self._sendSMS(mobileTo = f'''+45{self.currentSelectedOrder['mobile']}''', msg = smsMsg) 
        
        #Update the new deadline time in the json file
        
        filePath = self.getJsonFilePathFromOrderId(self.currentSelectedOrder['id'])
        order = self.readJsonFile(filePath)
        order['order']['deliveryTime'] = newDeadlineTime
        order['notifyDelay'] = True
        self.saveJsonFile(dataDict = order, filePath = filePath)
        self.update()
        
    def _genSMSMessage(self, customerName, newDeadlineTime):
        
        msg = f'''Dear {customerName},
        
We are currently busy with our dimsum takeaway, and therefore expect a delay in delivering your order. We expect to deliver you order at around {newDeadlineTime}. 
        
This sms cannot be replied.
        
Best regards,
Hidden Dimsum 2900'''
        return msg
    
    def _getNewDeadline(self, delayMinutes, currentDeadline):
        #Get the new deadline time by addming delayMinutes to the current deadline
        today = datetime.date.today()
        deadline = today.strftime('%d-%m-%Y') + ' ' + currentDeadline
        deadline = datetime.datetime.strptime(deadline, '%d-%m-%Y %H:%M')
        newDeadline = deadline + datetime.timedelta(minutes = delayMinutes)
        newDeadlineTime = newDeadline.strftime('%H:%M')
        return newDeadlineTime
    
    def _sendSMS(self, mobileTo, msg):
        smsCred = self.cred['SMS']
        
        # Your Account SID from twilio.com/console
        account_sid = smsCred['account_sid']
        
        # Your Auth Token from twilio.com/console
        auth_token  = smsCred['auth_token']
        
        client = Client(account_sid, auth_token)

        message = client.messages.create(
            to= mobileTo,
            from_= smsCred['smsNumberFrom'],
            body=msg)
        
    def update_btn(self, b):
        self.update()

    def update(self):
        #Set the html box to blank
        self.smsHTML.value = ''
        self.delayMinutes.value = 0
        self.refreshJson()
        self.getOrderDataFrame()
        self.displayTable()
        self.displayMap()
        
    def refreshJson(self):
        '''
        Used for refresh all the latest json files
        '''
        files = glob.glob(self.takeawayFolderPath + '/*.json')
        self.orders = list()
        for fileName in files:
            order = self.readJsonFile(fileName)
            
            #Check if the order is a delivery
            if order['order']['delivery']:
                #Check if the key Complete exists 
                if 'Complete' not in order:
                    tmp = dict()
                    tmp['id'] = order['order']['id']
                    tmp['fullName'] = order['order']['fullName']
                    tmp['email'] = order['order']['email']
                    tmp['mobile'] = order['order']['mobile']
                    tmp['deliveryAddress'] = order['order']['deliveryAddress']
                    tmp['latitude'] = order['order']['latitude']
                    tmp['longitude'] = order['order']['longitude']
                    tmp['comments'] = order['order']['comments']
                    tmp['deadline'] = order['order']['deliveryTime']

                    self.orders.append(tmp)
        
    def getOrderDataFrame(self):
        '''
        Convert the content in self.orders into a pandas data frame 
        '''
        #The data frame contain the following columns
        self.orderDF = dict()
        self.orderDF['Order Id'] = list()
        self.orderDF['Name'] = list()
        self.orderDF['Address'] = list()
        self.orderDF['Mobile'] = list()
        self.orderDF['Comments'] = list()
        self.orderDF['Deadline'] = list()
        for order in self.orders:
            self.orderDF['Order Id'].append(order['id'])
            self.orderDF['Name'].append(order['fullName'])
            self.orderDF['Address'].append(order['deliveryAddress'])
            self.orderDF['Mobile'].append(order['mobile'])
            self.orderDF['Comments'].append(order['comments'])
            self.orderDF['Deadline'].append(order['deadline'])
                
        self.orderDF = pd.DataFrame.from_dict(self.orderDF)
    
    def getGeoCoordinateFromOrderId(self, orderId):
        for order in self.orders:
            if order['id'] == orderId:
                latitude = order['latitude']
                longitude = order['longitude']
                deadline = order['deadline']
                return latitude, longitude, deadline
    
    def getOrderIdFromGeoCoordinate(self, coordinate):
        '''
        Given the latitude and longitude the method returns the order id
        '''
        for order in self.orders:
            if order['latitude'] == str(coordinate[0]) and order['longitude'] == str(coordinate[1]):
                return order
    
    def readJsonFile(self, filePath):
        with open(filePath,'r') as fileId:
            data = json.load(fileId)
        return data
    
    def saveJsonFile(self, dataDict, filePath):
        '''
        Given the dictionary dataDict and the filePath, this method saves it to json file
        '''
        with open(filePath, 'w') as fileId:
            json.dump(dataDict, fileId)

hd2900Coordinate = (55.73228810541183, 12.575497656450752)
deliveryRadius = 8
deliveryRadius = deliveryRadius * 1000       

In [2]:
GeoMap = GeoMapping(takeawayFolderPath, hd2900Coordinate, deliveryRadius)